In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
import sys

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [6]:
data_date = "2018_05_29_15_09_37"
data_dir = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-3/2018_05_29/" + data_date

In [7]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
params.print()

requestedTRamp 1200.0
configuredTRamp 1200.0
startFrequency 5725000000.0
stopFrequency 5875000000.0
frequencyBand 150000000.0
frequencyDevReq 50000.0
frequencyDevConf 50001.1
numberOfSteps 3000
waveformType 1
rampsPerBuffer 375
bufferSize 9000000.0
buffersPerFile 2
rampsPerFile 750
fileSize 18000000.0
samplingFrequency 10000000.0
samplesPerRamp 12000
skipNSamples 0


In [8]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 71
nbFiles = 60
lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

buffersPerFile = 2, samplesPerRamp = 12000, rampsPerFile = 750


# Read the data

In [9]:
scalingFactor = 2 / 65535
offset = -32768

In [10]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

# Read all files

In [11]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

k = 0, block = 142 / 260
k = 1, block = 144 / 260
k = 2, block = 146 / 260
k = 3, block = 148 / 260
k = 4, block = 150 / 260
k = 5, block = 152 / 260
k = 6, block = 154 / 260
k = 7, block = 156 / 260
k = 8, block = 158 / 260
k = 9, block = 160 / 260
k = 10, block = 162 / 260
k = 11, block = 164 / 260
k = 12, block = 166 / 260
k = 13, block = 168 / 260
k = 14, block = 170 / 260
k = 15, block = 172 / 260
k = 16, block = 174 / 260
k = 17, block = 176 / 260
k = 18, block = 178 / 260
k = 19, block = 180 / 260
k = 20, block = 182 / 260
k = 21, block = 184 / 260
k = 22, block = 186 / 260
k = 23, block = 188 / 260
k = 24, block = 190 / 260
k = 25, block = 192 / 260
k = 26, block = 194 / 260
k = 27, block = 196 / 260
k = 28, block = 198 / 260
k = 29, block = 200 / 260
k = 30, block = 202 / 260
k = 31, block = 204 / 260
k = 32, block = 206 / 260
k = 33, block = 208 / 260
k = 34, block = 210 / 260
k = 35, block = 212 / 260
k = 36, block = 214 / 260
k = 37, block = 216 / 260
k = 38, block = 218 / 

In [12]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [13]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

**Set the shifted flag properly**

In [14]:
rampDownFirst = 1

## Spectrum

### No window

In [15]:
samplesPerDownRamp = int(samplesPerRamp/2)

In [16]:
ifft = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp], axis=1)

In [ ]:
#ifft_abs = 20 * np.log10( np.abs( ifft ) )

In [ ]:
if 0:
    plt.figure()
    plt.imshow( ifft_abs[::100, 100:1000], cmap='jet' )
    title = "no window " + data_date
    plt.title(title)
    plt.savefig( data_dir + "/" + title + ".png" )

In [ ]:
plt.figure()
for k in range(10):
    plt.plot( 20 * np.log10( np.abs( np.fft.fft( ifft[:,k] ) ) ), label="{}".format(k) )
plt.legend()
plt.grid()
plt.title(data_date)

In [17]:
plt.figure()
for k in range(10):
    plt.plot( np.abs( ifft[:,k] ), label="{}".format(k) )
plt.legend()
plt.grid()
plt.title(data_date)

### Hanning

In [ ]:
hanning = np.hanning(samplesPerDownRamp)

In [ ]:
fft_A_hanning = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*hanning, axis=1)

In [ ]:
fft_abs_hanning = 20 * np.log10( np.abs( fft_A_hanning ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_hanning[::100, 0:1000], cmap='jet' )
title = "hanning " + data_date
plt.title(title)
plt.savefig( data_dir + "/" + title + ".png" )

### Kaiser

In [ ]:
# beta Window shape
# 0    Rectangular
# 5    Similar to a Hamming
# 6    Similar to a Hanning
# 8.6  Similar to a Blackman
# kaiser = np.kaiser(samplesPerDownRamp)

In [ ]:
fft_A_kaiser = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*kaiser, axis=1)

In [ ]:
fft_abs_kaiser = 20 * np.log10( np.abs( fft_A_kaiser ) )

In [ ]:
plt.figure()
plt.imshow( fft_abs_kaiser[::100, 0:1000], cmap='gray' )
plt.title("kaiser " + data_date)

# Build RD

In [36]:
#RDc = build_rd_from_data( params, A_reshaped, shifted )
withHanning = 0
RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )

shape of the samples matrix = (45000, 12000)
ramp down first in the data files


In [37]:
coupling_RD = np.average(RDc, 0)
if withHanning:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown_hanning".format(firstFile, lastFile), coupling_RD )
else:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown".format(firstFile, lastFile), coupling_RD )

In [38]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [39]:
if withHanning:
    np.save(data_dir + '/RD_files_{}_{}_rampDown_hanning'.format(firstFile, lastFile), RDc) 
else:
    np.save(data_dir + '/RD_files_{}_{}_rampDown'.format(firstFile, lastFile), RDc)

In [42]:
RD1 = np.fft.ifft( RDc-coupling_RD )

In [43]:
plt.figure()
plt.imshow(20 * np.log10( np.abs( RD1[::100, 0:1000] ) ) )